In [1]:
import pandas as pd 
import numpy as np 
import matplotlib.pyplot as plt 
from sklearn.datasets import load_breast_cancer
from sklearn.model_selection import train_test_split

/home/ahmad/.local/lib/python3.10/site-packages/matplotlib/projections/__init__.py:63: UserWarning: Unable to import Axes3D. This may be due to multiple versions of Matplotlib being installed (e.g. as a system package and as a pip package). As a result, the 3D projection is not available.
  warnings.warn("Unable to import Axes3D. This may be due to multiple versions of "


In [9]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split

# Supaya hasil acaknya sama terus
np.random.seed(42)

# Bikin 1000 Data Pelanggan
n_samples = 1000

data = {
    'Tenure': np.random.randint(1, 72, n_samples), # 1 s/d 72 bulan
    'MonthlyCharges': np.random.randint(20, 100, n_samples), # 20 s/d 100 dollar
    'PaperlessBilling': np.random.randint(0, 2, n_samples),
    'SeniorCitizen': np.random.randint(0, 2, n_samples),
    'Partner': np.random.randint(0, 2, n_samples),
    'Dependents': np.random.randint(0, 2, n_samples),
    'PhoneService': np.random.randint(0, 2, n_samples),
    'LuckyNumber': np.random.randint(1, 1000, n_samples) # Fitur Sampah (Random)
}

df = pd.DataFrame(data)

# Bikin kolom TotalCharges (Tenure * Monthly) + sedikit acak
df['TotalCharges'] = df['Tenure'] * df['MonthlyCharges']

# Bikin Target CHURN (Logika: Kalau Tenure kecil & Tagihan Mahal -> Cenderung Churn)
# Ini rumus rahasia biar ada polanya
score = (100 - df['Tenure']) + (df['MonthlyCharges'] * 0.5) + np.random.normal(0, 20, n_samples)
df['Churn'] = (score > 80).astype(int) # 1 = Churn, 0 = Setia
df

,Tenure,MonthlyCharges,PaperlessBilling,SeniorCitizen,Partner,Dependents,PhoneService,LuckyNumber,TotalCharges,Churn
0,52,65,0,1,0,1,1,963,3380,0
1,15,35,0,0,0,0,0,193,525,1
2,61,87,0,0,0,0,0,895,5307,0
3,21,56,1,1,1,0,1,58,1176,1
4,24,73,0,1,1,0,1,437,1752,1
...,...,...,...,...,...,...,...,...,...,...
995,51,99,1,1,0,0,1,352,5049,1
996,8,26,0,0,0,1,1,701,208,1
997,34,71,1,1,1,1,0,928,2414,1
998,35,33,0,0,1,0,1,256,1155,1


In [10]:
x = df.drop(columns='Churn')
y = df['Churn']

In [13]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size = 0.2, random_state=42)

In [23]:
from sklearn.feature_selection import SelectKBest, chi2
selector_filter = SelectKBest(score_func=chi2, k=3)

#fit dan transform 
x_train_filter = selector_filter.fit_transform(x_train, y_train)
x_test_filter = selector_filter.transform(x_test)

cols_filter = selector_filter.get_support(indices=True)
x.columns[cols_filter]

Index(['Tenure', 'MonthlyCharges', 'TotalCharges'], dtype='object')

In [50]:
from sklearn.feature_selection import RFE 
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
x_train_scaled = scaler.fit_transform(x_train)
x_test_scaled = scaler.transform(x_test)

model_lr = LogisticRegression(solver='liblinear')

#siapkan rfe
selector_rfe = RFE(estimator=model_lr, n_features_to_select=3)

#fit dan transform
x_train_rfe = selector_rfe.fit_transform(x_train_scaled, y_train)
x_test_rfe = selector_rfe.transform(x_test_scaled)

cols_rfe = selector_rfe.get_support(indices=True)
x.columns[cols_rfe]

Index(['Tenure', 'MonthlyCharges', 'TotalCharges'], dtype='object')

In [43]:
from sklearn.ensemble import RandomForestClassifier

model_rf = RandomForestClassifier(n_estimators=100, random_state=42)
model_rf.fit(x_train, y_train)

#ambil skor kepentingan
importance = model_rf.feature_importances_
indices_rf = np.argsort(importance)[::-1][:3]
indices_rf

array([0, 8, 1])

In [48]:
x_train_emb = x_train.iloc[:, indices_rf]
x_test_emb = x_test.iloc[:, indices_rf]
x_train_emb

,Tenure,TotalCharges,MonthlyCharges
29,64,3584,56
535,15,465,31
695,7,518,74
557,17,918,54
836,6,270,45
...,...,...,...
106,15,885,59
270,46,1288,28
860,47,3102,66
435,37,2664,72
